In [2]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn

In [3]:
def bpr_K(y_pred_BD, y_true_BD, K=4, perturbed_top_K_func=None):

    top_K_ids_BKD = perturbed_top_K_func(y_pred_BD)
    # Sum over k dim
    top_K_ids_BD = top_K_ids_BKD.sum(dim=-2)

    true_top_K_val_BD, _  = torch.topk(y_true_BD, K) 
    denominator_B = torch.sum(true_top_K_val_BD, dim=-1)
    numerator_B = torch.sum(top_K_ids_BD * y_true_BD, dim=-1)
    bpr_B = numerator_B/denominator_B

    return bpr_B

In [ ]:
    b, d = x.shape
    
    # Generate Gaussian noise with specified number of samples and standard deviation
    noise = torch.normal(mean=0.0, std=1.0, size=(b, num_samples, d)).to(x.device)

In [5]:
class PerturbedTopK(nn.Module):

    def __init__(self, k: int, noise: torch.tensor, sigma: float = 0.05, num_samples: int = 500):
        super(PerturbedTopK, self).__init__()
    
        self.noise = noise
        self.k = k
        self.sigma = sigma
        self.num_samples = num_samples
    
    def __call__(self, x):
        # Return the output of the PerturbedTopKFunction, applied to the input tensor
        # using the k, num_samples, and sigma attributes as arguments
        return PerturbedTopKFunction.apply(x, self.k, self.noise, self.sigma, self.num_samples)

class PerturbedTopKFunction(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, x, k: int, noise: torch.tensor, sigma: float = 0.05, num_samples: int = 500):
    
        b, d = x.shape
 
        # Add noise to the input tensor
        perturbed_x = x[:, None, :] + noise * sigma # b, nS, d
        
        # Perform top-k pooling on the perturbed tensor
        topk_results = torch.topk(perturbed_x, k=k, dim=-1, sorted=False)
        
        # Get the indices of the top k elements
        indices = topk_results.indices # b, nS, k
        
        # Sort the indices in ascending order
        indices = torch.sort(indices, dim=-1).values # b, nS, k

        # Convert the indices to one-hot tensors
        perturbed_output = torch.nn.functional.one_hot(indices, num_classes=d).float()
        
        # Average the one-hot tensors to get the final output
        indicators = perturbed_output.mean(dim=1) # b, k, d

        # Save constants and tensors for backward pass
        ctx.k = k
        ctx.num_samples = num_samples
        ctx.sigma = sigma

        ctx.perturbed_output = perturbed_output
        ctx.noise = noise

        return indicators


    @staticmethod
    def backward(ctx, grad_output):
        # If there is no gradient to backpropagate, return tuple of None values
        if grad_output is None:
            return tuple([None] * 5)

        noise_gradient = ctx.noise
        
        # Calculate expected gradient
        expected_gradient = (
            torch.einsum("bnkd,bne->bkde", ctx.perturbed_output, noise_gradient)
            / ctx.num_samples
            / ctx.sigma
        ) * float(ctx.k)
        
        grad_input = torch.einsum("bkd,bkde->be", grad_output, expected_gradient)
        
        return (grad_input,) + tuple([None] * 5)

In [8]:
# batch dimension
B = 100
# feature dimension
D = 10
# top-K
K=4
rating_BD = torch.rand(B, D)
labels_BD = torch.rand(B, D)


sigma = 0.05
num_samples=500

    
# Generate Gaussian noise with specified number of samples and standard deviation
noise = torch.normal(mean=0.0, std=1.0, size=(B, num_samples, D)).to(rating_BD.device)

perturbed_top_K_func = PerturbedTopK(k=K, noise=noise, num_samples=num_samples, sigma=sigma)



In [9]:
def loss(rating_BD):
    """Compute loss. Labels are not an input so we can just get gradient of rating"""
    return -bpr_K(rating_BD, labels_BD, perturbed_top_K_func=perturbed_top_K_func, K=K)

In [20]:
# Goal: gradient of loss w.r.t. rating_BD for each B separately
grad_loss_TD = torch.autograd.functional.jacobian(loss, (rating_BD), vectorize=True,  strategy='forward-mode')

NotImplementedError: You must implement the jvp function for custom autograd.Function to use it with forward mode AD.

In [15]:
x = tf.Variable(2.0)
with tf.GradientTape() as tape:
  y0 = x**2
  y1 = 1 / x

print(tape.gradient({'y0': y0, 'y1': y1}, x).numpy())

3.75


2024-07-05 15:20:17.475711: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-05 15:20:17.475745: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: p1cmp078.pax.tufts.edu
2024-07-05 15:20:17.475754: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: p1cmp078.pax.tufts.edu
2024-07-05 15:20:17.475804: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.129.3
2024-07-05 15:20:17.475830: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.129.3
2024-07-05 15:20:17.475835: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.129.3


In [14]:
import tensorflow as tf

2024-07-05 15:19:46.649358: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-05 15:19:46.706224: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-05 15:19:46.706251: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-05 15:19:46.707330: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-05 15:19:46.714907: I tensorflow/core/platform/cpu_feature_guar

In [19]:
x = tf.Variable([2.,3.])

with tf.GradientTape() as tape:
  y = x * [3., 4.]

print(tape.batch_jacobian(y, x,).numpy())

ValueError: Shape (2,) must have rank at least 2